## Making Use of the **PACE** MODEL

## Plan

## Analyze: Statistical Analysis with Python

## Construct: Detailed report with Tableau

## Execute: Presenting the report online

## **PLAN** phase

- The ELT process will be executed
- Data will be cleaned for the Analyze phase

E - Data extracted from Kaggle

L - Data to be loaded to a sqlserver Database

T - Data will be transformed. From JSON to table format and cleaned

ELT PROCESS

L -- Loading JSON data

In [28]:
Declare @JSON varchar(max)
SELECT @JSON=BulkColumn
FROM OPENROWSET (BULK 'G:\My Drive\PortfolioProjects\flipkart_fashion_products_dataset.json\flipkart_fashion_products_dataset.json', SINGLE_CLOB) import
SELECT * 
INTO Flipkart_data
FROM OPENJSON (@JSON)
WITH
( 
    _id NVARCHAR(MAX),
    actual_price VARCHAR(50),
    average_rating VARCHAR(10),
    brand VARCHAR(50),
    category VARCHAR(100),
    crawled_at VARCHAR(100),
    description VARCHAR(MAX),
    discount VARCHAR(20),
    -- images VARCHAR(MAX),
    out_of_stock VARCHAR(10),
    pid VARCHAR(50),
    product_details nvarchar(max) as JSON,
    seller VARCHAR(100),
    selling_price VARCHAR(50),
    sub_category VARCHAR(100),
    title VARCHAR(200)
);

(30000 rows affected)

Total execution time: 00:00:54.423

In [17]:
SELECT TOP 5 *
FROM Flipkart_data;

(5 rows affected)

Total execution time: 00:00:00.060

_id,actual_price,average_rating,brand,category,crawled_at,description,discount,out_of_stock,pid,product_details,seller,selling_price,sub_category,title
a549a2af-504f-59bd-ae23-7acb21ec5d11,"1,539",2.7,,Clothing and Accessories,"02/10/2021, 20:12:40","We NextEdgeRetails are a professional tie manufacturer. We focus on ties for many years and In our store, we have many other styles (neckties,bow ties,tie sets,etc.) for sell which are bound to meet your needs, you can click our brand name nu-Lite to choose. Our products are famous for their comfort, durability and design because of the reliable quality, our products have been highly favored of the majority of customers. Besides they also would be perfect gifts or excellent additions to your wardrobe as well as for all fashion forward men, this tie is going to be a perfect pick. Make your Monday meetings more fun with this tie and show the world your style. Wearing an elegant man tie is an effective way to make you look charming and energetic. Be stylish and make a new style statement with nu-Lite that includes a great range of formal/casual slim neckties with pocket square for men. These ties looks good with most shirts, winning you compliments wherever you go and it. Comes with elegant primary packing of wooden box, it's a great choice for gifting purpose. Material: Wrinkle-resistant with ultra-fine POLYESTER microfiber, looks and feels like silk and even more durable OCCASIONS: nu-Lite ties collection are great choices for formal, casual and less formal occasions, such as office events, dinner dates, and simple day to day activities and also for Wedding, Engagement, Prom Ball Party, Business, Office, Meeting, Birthday Gifts, Holiday Presents, Thanks Giving, Christmas, Events Dressing. HOW TO CARE: Dry Cleaning Recommended and Before the first use please use Low Temperature Ironing to smooth the wrinkles. NOTE: In case you don't like the ties simply send them back, our team is here for any question.",54% off,false,CTPFVZHVZVU4VQW9,"[ { ""Model Name"": ""Men's Micro Polyester Checkered Necktie Set with Pocket Square and Lapel Pin"" }, { ""Style Code"": ""NL-Check-15"" }, { ""Other Features"": ""AMAZING GIFT IDEA: Comes inside a wooden box, These nu-Lite stylish ties for men can make the perfect gift for every man and for every occasion, so get this stunning necktie for your husband for your anniversary, your dad for Father\u2019s Day, your brother for his birthday, your son for Christmas, or your boyfriend for Valentine\u2019s Day and on many more occasion like Daily Dress, Birthday, Wedding, Engagement, Party, Business, Office, Meeting etc. CARE: Dry clean only. Never place ties in washer or dryer as this will ruin the fabric and damage the sewing of the tie. High density fabric,No fade,wrinkle resistance and easy to take care of."" }, { ""Sales Package"": ""1 Tie with other accessories."" } ]",NextEdgeRetails,699,Clothing Accessories,nu-Lite Satin Tie & Cufflink (Grey)
8d92559c-6538-5f0c-bc6a-a880207b789d,"3,299",2.7,,Clothing and Accessories,"02/10/2021, 20:12:40","Wearing an elegant man tie is an effective way to make you look charming and energetic. nu-Lite includes an amazing range of formal/casual slim neckties with pocket square for men. Further the classy & stylish appeal of these ties will certainly appeal to all & is bound to turn all heads. Adding complementing pocket square will enhance overall attire. It looks great with any dress or button up shirt. We have an enormous selection of ties to choose from, you can find the most elegant necktie designs and colors for each season of the year. These ties looks good with most shirts, winning you compliments wherever you go. Comes with elegant primary packing, great choice for gifting purpose. Each set comes with:",81% off,false,CTPFVZT6GAFZ7BJX,"[ { ""Model Name"": ""Formal/Casual Printed Polyester Necktie Set with Pocket Square, Lapel Pin and Cufflinks for Men"" }, { ""Style Code"": ""nu-Lite_Tie_09"" }, { ""Other Feature

In [19]:
-- HAVING A LOOK AT PRODUCT DETAILS
SELECT top 5 _id,
    JSON_VALUE(detail.value, '$."Style Code"') AS StyleCode,
    JSON_VALUE(detail.value, '$."Closure"') AS Closure,
    JSON_VALUE(detail.value, '$."Pockets"') AS Pockets,
    JSON_VALUE(detail.value, '$."Fabric"') AS Fabric,
    JSON_VALUE(detail.value, '$."Pattern"') AS Pattern,
    JSON_VALUE(detail.value, '$."Color"') AS Color
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail;

(5 rows affected)

Total execution time: 00:00:00.034

_id,StyleCode,Closure,Pockets,Fabric,Pattern,Color
a549a2af-504f-59bd-ae23-7acb21ec5d11,NULL,NULL,NULL,NULL,NULL,NULL
a549a2af-504f-59bd-ae23-7acb21ec5d11,NL-Check-15,NULL,NULL,NULL,NULL,NULL
a549a2af-504f-59bd-ae23-7acb21ec5d11,NULL,NULL,NULL,NULL,NULL,NULL
a549a2af-504f-59bd-ae23-7acb21ec5d11,NULL,NULL,NULL,NULL,NULL,NULL
8d92559c-6538-5f0c-bc6a-a880207b789d,NULL,NULL,NULL,NULL,NULL,NULL


Unnesting the Product\_details into a seperate table

In [22]:

-- FOR each Subset of Product details, we will reiterate the same query and store in a table
-- This query will produce two columns for for id and the other for the subset
-- Null values will be removed

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Style Code"') AS StyleCode
INTO Stylecode
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Style Code"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Closure"') AS Closure
INTO Closure
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Closure"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Pockets"') AS Pockets
INTO Pockets
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Pockets"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Fabric"') AS Fabric
INTO Fabric
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Fabric"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Pattern"') AS Pattern
INTO Pattern
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Pattern"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Color"') AS Color
INTO Color
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Color"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Sales Package"') AS [Sales Package]
INTO Sales_package
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Sales Package"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Fabric Care"') AS [Fabric Care]
INTO Fabric_care
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Fabric Care"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Type"') AS [Type]
INTO Type
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Type"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Sleeve"') AS [Sleeve]
INTO Sleeve
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Sleeve"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Size"') AS [Size]
INTO Size
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Size"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Brand Color"') AS [Brand Color]
INTO Brand_Color
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Brand Color"') IS NOT NULL;

SELECT 
    _id as id,
    JSON_VALUE(detail.value, '$."Pack of"') AS [Pack of]
INTO Pack_of
FROM Flipkart_data
CROSS APPLY OPENJSON(PRODUCT_DETAILS) AS detail
WHERE JSON_VALUE(detail.value, '$."Pack of"') IS NOT NULL;



(27568 rows affected)

(4550 rows affected)

(1778 rows affected)

(27804 rows affected)

(27507 rows affected)

...

(20411 rows affected)

(13420 rows affected)

(14029 rows affected)

(19417 rows affected)

Total execution time: 00:00:53.620

In [29]:
/*
Using Coalesce to replace null ids with other ids.
Using MAX to aggregate the values for each column while grouping by the Id column. 
This will combine all the records with the same ID into a single row, 
and each column will contain the maximum non-null value from the individual rows with the same ID.
If this is not done, there will be duplicate product_ids, with each id showing only 1 detail
*/

SELECT
    COALESCE(st.id, cl.id, p.id, f.id, pa.id,
                co.id, sp.id, fc.id, t.id, sv.id, sz.id, bc.id, po.id) AS Id,
    MAX(StyleCode) AS StyleCode,
    MAX(Closure) AS Closure,
    MAX(Pockets) AS Pockets,
    MAX(Fabric) AS Fabric,
    MAX(Pattern) AS Pattern,
    MAX(Color) AS Color,
    MAX([Type]) AS [Type],
    MAX([Sales Package]) AS [Sales Package],
    MAX([Fabric Care]) AS [Fabric Care],
    MAX([Size]) AS [Size],
    MAX(Sleeve) AS Sleeve,
    MAX([Brand Color]) AS [Brand Color],
    MAX([Pack of]) AS [Pack of]
INTO flipkart_products
FROM
    Stylecode st
FULL OUTER JOIN Closure cl ON st.id = cl.id
FULL OUTER JOIN Pockets p ON p.id = COALESCE(st.id, cl.id)
FULL OUTER JOIN Fabric f ON f.id = COALESCE(st.id, cl.id, p.id)
FULL OUTER JOIN Pattern pa ON pa.id = COALESCE(st.id, cl.id, p.id, f.id)
FULL OUTER JOIN Color co ON co.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id)
FULL OUTER JOIN Sales_package sp ON sp.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id)
FULL OUTER JOIN Fabric_care fc ON fc.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id)
FULL OUTER JOIN [Type] t ON t.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id, fc.id)
FULL OUTER JOIN Sleeve sv ON sv.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id, fc.id, t.id)
FULL OUTER JOIN [Size] sz ON sz.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id, fc.id, t.id, sv.id)
FULL OUTER JOIN [Brand_Color] bc ON bc.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id, fc.id, t.id, sv.id, sz.id)
FULL OUTER JOIN [Pack_of] po ON po.id = COALESCE(st.id, cl.id, p.id, f.id, pa.id, co.id, sp.id, fc.id, t.id, sv.id, sz.id, bc.id)
GROUP BY
    COALESCE(st.id, cl.id, p.id, f.id, pa.id,
                co.id, sp.id, fc.id, t.id, sv.id, sz.id, bc.id, po.id)
-- ORDER BY 1
;


Warning: Null value is eliminated by an aggregate or other SET operation.

(29985 rows affected)

Total execution time: 00:00:11.144

## CLEANING

In this stage, we will clean the flipkart\_data table

In [31]:
-- UPDATE
UPDATE Flipkart_data
SET actual_price = TRIM(REPLACE(actual_price, ',', ''));

SELECT TOP 10 actual_price
FROM Flipkart_data

(30000 rows affected)

(10 rows affected)

Total execution time: 00:00:00.600

actual_price
1539
3299
3299
1539
1539
1539
2999
1539
1539
1539


In [32]:
-- UPDATE
UPDATE Flipkart_data
SET selling_price = TRIM(REPLACE(selling_price, ',', ''));

SELECT TOP 5 selling_price
FROM Flipkart_data

(30000 rows affected)

(5 rows affected)

Total execution time: 00:00:00.543

selling_price
699
599
899
349
349


In [55]:
-- Changing the datatypes to int
ALTER TABLE flipkart_data
ALTER COLUMN actual_price int;

ALTER TABLE flipkart_data
ALTER COLUMN selling_price int;

Commands completed successfully.

Total execution time: 00:00:01.664

### There are a lot of rows with empty strings in the **Average\_rating** column.  
Those cells need to be filled.  
Checking the MODE, MEAN AND MEDIAN to determine what to fill it with

In [33]:
-- MEAN
SELECT round(avg(CAST(average_rating as decimal(5,2))),1) as average
FROM Flipkart_data
WHERE --Excluding the empty string cells 
    (average_rating IS NOT NULL) 
    AND (average_rating <> '' OR average_rating <> LTRIM(RTRIM(average_rating)));

(1 row affected)

Total execution time: 00:00:00.180

average
3.600000


In [71]:
-- MODE
SELECT average_rating,COUNT(1) frequency
FROM Flipkart_data
WHERE (average_rating IS NOT NULL) 
    AND (average_rating <> '' OR average_rating <> LTRIM(RTRIM(average_rating)))
GROUP BY average_rating
ORDER BY 2 DESC

(38 rows affected)

Total execution time: 00:00:00.091

average_rating,frequency
3.9,2690
3.8,2532
3.6,1957
4.2,1838
4,1749
3.7,1562
3.4,1506
4.1,1459
3.2,1430
4.5,1383


In [69]:
-- CHECKING MEDIAN
-- WE HAVE 27,554 NON BLANK RECORDS. THIS QUERY SHOWS THE NUMBER AT THE MIDDLE
SELECT average_rating
FROM Flipkart_data
WHERE (average_rating IS NOT NULL) 
    AND (average_rating <> '' OR average_rating <> LTRIM(RTRIM(average_rating)))
ORDER BY 1
OFFSET 13770 rows
FETCH NEXT 10 ROWS ONLY;

(10 rows affected)

Total execution time: 00:00:00.214

average_rating
3.8
3.8
3.8
3.8
3.8
3.8
3.8
3.8
3.8
3.8


In [34]:
-- Updating the empty strings with 3.8. Which was the mode and median
UPDATE Flipkart_data
SET average_rating = 3.8
WHERE average_rating IS NULL OR LTRIM(RTRIM(average_rating)) = '';




(2446 rows affected)

Total execution time: 00:00:00.152

In [86]:
-- A look at the updated columns
SELECT average_rating,COUNT(1) frequency
FROM Flipkart_data
WHERE (average_rating IS NOT NULL) 
    AND (average_rating <> '' OR average_rating <> LTRIM(RTRIM(average_rating)))
GROUP BY average_rating
ORDER BY 2 DESC

(38 rows affected)

Total execution time: 00:00:00.105

average_rating,frequency
3.80,4978
3.90,2690
3.60,1957
4.20,1838
4.00,1749
3.70,1562
3.40,1506
4.10,1459
3.20,1430
4.50,1383


In [35]:
-- Change data type from varchar to decimal
ALTER TABLE flipkart_data
ALTER COLUMN average_rating decimal;

Commands completed successfully.

Total execution time: 00:00:00.582

### Cleaning empty strings in the **brand** column

In [ ]:
-- Update rows in table to reflect change
UPDATE Flipkart_data
SET brand = 'Not Stated' -- All empty strings will be changed to Not Stated
WHERE brand IS NULL OR LTRIM(RTRIM(brand)) = '';

-- Removing any white spaces at the left and right of each string
UPDATE Flipkart_data
SET brand = LTRIM(RTRIM(brand))

## Empty strings in the **discount** column

In [71]:
/*
I will assume that empty cells in the
discount table means that no discount was offered 
*/

UPDATE Flipkart_data
SET discount = 'None' -- All empty strings will be changed to None
WHERE discount IS NULL OR LTRIM(RTRIM(discount)) = '';

(941 rows affected)

Total execution time: 00:00:00.239

## Empty strings in the **seller** column

In [72]:
UPDATE Flipkart_data
SET seller = 'Anonymous' -- All empty strings will be changed to Anonymous
WHERE seller IS NULL OR LTRIM(RTRIM(seller)) = '';

(1741 rows affected)

Total execution time: 00:00:00.234

In [73]:
-- Confirm the data type

SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'flipkart_data';


(15 rows affected)

Total execution time: 00:00:00.055

COLUMN_NAME,DATA_TYPE
_id,nvarchar
actual_price,int
average_rating,decimal
brand,varchar
category,varchar
crawled_at,varchar
description,varchar
discount,varchar
out_of_stock,varchar
pid,varchar


CLEANING THE flipkart\_products\_details table

### There are a lot of empty records.

### Since these are categorical fields - with \[Pack Of\] being an exception, the records will be filled with a categorical flag of 'Not Stated'

In [20]:
UPDATE flipkart_products
SET StyleCode = ISNULL(RTRIM(LTRIM(StyleCode)), 'Not Stated');

UPDATE flipkart_products
SET Closure = ISNULL(RTRIM(LTRIM(Closure)), 'Not Stated');

UPDATE flipkart_products
SET Pockets = ISNULL(RTRIM(LTRIM(Pockets)), 'Not Stated');

UPDATE flipkart_products
SET Fabric = ISNULL(RTRIM(LTRIM(Fabric)), 'Not Stated');

UPDATE flipkart_products
SET Pattern = ISNULL(RTRIM(LTRIM(Pattern)), 'Not Stated');

UPDATE flipkart_products
SET Color = ISNULL(RTRIM(LTRIM(Color)), 'Not Stated');

UPDATE flipkart_products
SET [Type] = ISNULL(RTRIM(LTRIM([Type])), 'Not Stated');

UPDATE flipkart_products
SET [Fabric Care] = ISNULL(RTRIM(LTRIM([Fabric Care])), 'Not Stated');

UPDATE flipkart_products
SET [Size] = ISNULL(RTRIM(LTRIM([Size])), 'Not Stated');

UPDATE flipkart_products
SET [Sleeve] = ISNULL(RTRIM(LTRIM([Sleeve])), 'Not Stated');

UPDATE flipkart_products
SET [Brand Color] = ISNULL(RTRIM(LTRIM([Brand Color])), 'Not Stated');

(29985 rows affected)

(29985 rows affected)

(29985 rows affected)

(29985 rows affected)

(29985 rows affected)

...

(29985 rows affected)

(29985 rows affected)

(29985 rows affected)

(29985 rows affected)

Total execution time: 00:00:03.107

In [22]:
-- Take a look

SELECT TOP 20 * 
FROM flipkart_products

(20 rows affected)

Total execution time: 00:00:00.226

Id,StyleCode,Closure,Pockets,Fabric,Pattern,Color,Type,Sales Package,Fabric Care,Size,Sleeve,Brand Color,Pack of
00005493-1875-5b43-8061-20c7d0cbe6b1,AX-Basic-Silver Grey,Not Stated,Not Stated,Cotton Blend,Solid,Not Stated,Polo Neck,1 T-Shirt,"Regular Machine Wash, Do not Iron on print/embroidery/embellishment, Reverse and dry, Do not tumble dry, Dry in shade, Do not dry clean",M,Half Sleeve,Silver Grey,1
0003c6ae-73c1-5f18-a5e6-778e5f7b8804,T10000580,Not Stated,Not Stated,Cotton Lycra Blend,Solid,Not Stated,Round Neck,1 T-Shirt,Not Stated,XL,Half Sleeve,Aqua,1
000636e4-07c0-5d26-adfd-f6114389edb7,EKSH001982,Not Stated,Not Stated,Cotton Blend,Checkered,Khaki,Not Stated,NULL,Gentle Machine Wash,Not Stated,Full Sleeve,Not Stated,1
000cb2a8-7ae7-5905-a828-9a9ebc9947b8,(GRN-KWI-ORG)-3P,Not Stated,Not Stated,Polycotton,Solid,Not Stated,Polo Neck,3 T-Shirts,Regular Machine Wash,XXL,Short Sleeve,Green;Light Green;Orange,3
000cb961-8d64-513b-896a-ab29d85fb97d,META_TSHIRT_BL-6,Not Stated,Not Stated,Pure Cotton,Graphic Print,Not Stated,Round Neck,1,Regular Machine Wash,S,Half Sleeve,Black,1
000d890b-13a7-5fdd-b740-e7136806575c,AX-531-Navy,Not Stated,Not Stated,Cotton Blend,"Striped, Checkered",Blue,Not Stated,1 Sweatshirt,Do Not Iron on Print/Embroidery/Embellishment,Not Stated,Full Sleeve,Not Stated,NULL
000f6224-fa1b-5771-b73d-a1f21773101c,Not Stated,Not Stated,Not Stated,Cotton,Not Stated,Multicolor,Not Stated,NULL,Cotton,Not Stated,Not Stated,Not Stated,NULL
000fe678-23a2-595b-86bf-5c6ece756471,S100-C2-Rd_Gry-,Not Stated,Not Stated,Pure Cotton,"Self Design, Solid",Not Stated,Round Neck,NULL,Gentle Machine Wash,L,Short Sleeve,"Red, Grey",2
001153d6-f9aa-5759-b61f-1b4fb288c686,Guruji-Blue,Not Stated,Not Stated,Pure Cotton,Printed,Not Stated,Round Neck,1 t shirt,Gentle Machine Wash,L,Short Sleeve,Blue,1
0013bc06-90e6-5f5a-872a-2f4daa082ab1,GYM WORKOUT CUT SLEEVES HOODIE,Not Stated,Not Stated,Pure Cotton,Printed,Not Stated,Hooded Neck,1,Gentle Machine Wash,XL,Sleeveless,Black,1


### Updating the last 2 columns - Sales Package and Pack of.

When **Pack of field is null** **and Sales Package is not null ,** <span style="color: var(--vscode-foreground);">I'd like to get the first numeric value from the Sales Package field and replace it.</span>

In [55]:
UPDATE flipkart_products
SET [Pack of] =  CASE 
        WHEN PATINDEX('%[0-9]%', [Sales Package]) > 0
        THEN TRY_CAST(SUBSTRING([Sales Package], PATINDEX('%[0-9]%', [Sales Package]), 
                CHARINDEX(' ', [Sales Package] + ' ', PATINDEX('%[0-9]%', [Sales Package])) - PATINDEX('%[0-9]%', [Sales Package])) AS int)
        ELSE NULL
    END
WHERE [Pack of] IS NULL AND [Sales Package] IS NOT NULL;


(4735 rows affected)

Total execution time: 00:00:00.174

In [61]:
-- Checking to see if it worked
SELECT [Sales Package], [Pack of]
FROM flipkart_products
WHERE [Sales Package] IN ('VIP Frenchie-Plus-2', '6 Hunky Cotton Briefs')

(9 rows affected)

Total execution time: 00:00:00.094

Sales Package,Pack of
VIP Frenchie-Plus-2,2
6 Hunky Cotton Briefs,6
6 Hunky Cotton Briefs,6
VIP Frenchie-Plus-2,2
VIP Frenchie-Plus-2,2
VIP Frenchie-Plus-2,2
VIP Frenchie-Plus-2,2
6 Hunky Cotton Briefs,6
VIP Frenchie-Plus-2,2


In [63]:
-- Now Updating the null values in sales package

UPDATE flipkart_products
SET [Sales Package] = ISNULL(RTRIM(LTRIM([Sales Package])), 'Not Stated');

(29985 rows affected)

Total execution time: 00:00:00.196

In [66]:
-- and the still Null values in Pack of
UPDATE flipkart_products
SET [Pack of] = ISNULL(RTRIM(LTRIM([Pack of])), 'Not Stated');

(29985 rows affected)

Total execution time: 00:00:00.220

## Taking a Look at the cleaned tables

In [1]:
SELECT TOP 10 *
FROM Flipkart_data

(10 rows affected)

Total execution time: 00:00:00.157

Id,actual_price,average_rating,brand,category,crawled_at,description,discount,out_of_stock,pid,product_details,seller,selling_price,sub_category,title
a549a2af-504f-59bd-ae23-7acb21ec5d11,1539,3,Not Stated,Clothing and Accessories,"02/10/2021, 20:12:40","We NextEdgeRetails are a professional tie manufacturer. We focus on ties for many years and In our store, we have many other styles (neckties,bow ties,tie sets,etc.) for sell which are bound to meet your needs, you can click our brand name nu-Lite to choose. Our products are famous for their comfort, durability and design because of the reliable quality, our products have been highly favored of the majority of customers. Besides they also would be perfect gifts or excellent additions to your wardrobe as well as for all fashion forward men, this tie is going to be a perfect pick. Make your Monday meetings more fun with this tie and show the world your style. Wearing an elegant man tie is an effective way to make you look charming and energetic. Be stylish and make a new style statement with nu-Lite that includes a great range of formal/casual slim neckties with pocket square for men. These ties looks good with most shirts, winning you compliments wherever you go and it. Comes with elegant primary packing of wooden box, it's a great choice for gifting purpose. Material: Wrinkle-resistant with ultra-fine POLYESTER microfiber, looks and feels like silk and even more durable OCCASIONS: nu-Lite ties collection are great choices for formal, casual and less formal occasions, such as office events, dinner dates, and simple day to day activities and also for Wedding, Engagement, Prom Ball Party, Business, Office, Meeting, Birthday Gifts, Holiday Presents, Thanks Giving, Christmas, Events Dressing. HOW TO CARE: Dry Cleaning Recommended and Before the first use please use Low Temperature Ironing to smooth the wrinkles. NOTE: In case you don't like the ties simply send them back, our team is here for any question.",54% off,false,CTPFVZHVZVU4VQW9,"[ { ""Model Name"": ""Men's Micro Polyester Checkered Necktie Set with Pocket Square and Lapel Pin"" }, { ""Style Code"": ""NL-Check-15"" }, { ""Other Features"": ""AMAZING GIFT IDEA: Comes inside a wooden box, These nu-Lite stylish ties for men can make the perfect gift for every man and for every occasion, so get this stunning necktie for your husband for your anniversary, your dad for Father\u2019s Day, your brother for his birthday, your son for Christmas, or your boyfriend for Valentine\u2019s Day and on many more occasion like Daily Dress, Birthday, Wedding, Engagement, Party, Business, Office, Meeting etc. CARE: Dry clean only. Never place ties in washer or dryer as this will ruin the fabric and damage the sewing of the tie. High density fabric,No fade,wrinkle resistance and easy to take care of."" }, { ""Sales Package"": ""1 Tie with other accessories."" } ]",NextEdgeRetails,699,Clothing Accessories,nu-Lite Satin Tie & Cufflink (Grey)
8d92559c-6538-5f0c-bc6a-a880207b789d,3299,3,Not Stated,Clothing and Accessories,"02/10/2021, 20:12:40","Wearing an elegant man tie is an effective way to make you look charming and energetic. nu-Lite includes an amazing range of formal/casual slim neckties with pocket square for men. Further the classy & stylish appeal of these ties will certainly appeal to all & is bound to turn all heads. Adding complementing pocket square will enhance overall attire. It looks great with any dress or button up shirt. We have an enormous selection of ties to choose from, you can find the most elegant necktie designs and colors for each season of the year. These ties looks good with most shirts, winning you compliments wherever you go. Comes with elegant primary packing, great choice for gifting purpose. Each set comes with:",81% off,false,CTPFVZT6GAFZ7BJX,"[ { ""Model Name"": ""Formal/Casual Printed Polyester Necktie Set with Pocket Square, Lapel Pin and Cufflinks for Men"" }, { ""Style Code"": ""nu-Lite_Tie_09"" }, { ""Othe

In [2]:
SELECT TOP 10 *
FROM flipkart_products

(10 rows affected)

Total execution time: 00:00:00.058

Id,StyleCode,Closure,Pockets,Fabric,Pattern,Color,Type,Sales Package,Fabric Care,Size,Sleeve,Brand Color,Pack of
00005493-1875-5b43-8061-20c7d0cbe6b1,AX-Basic-Silver Grey,Not Stated,Not Stated,Cotton Blend,Solid,Not Stated,Polo Neck,1 T-Shirt,"Regular Machine Wash, Do not Iron on print/embroidery/embellishment, Reverse and dry, Do not tumble dry, Dry in shade, Do not dry clean",M,Half Sleeve,Silver Grey,1
0003c6ae-73c1-5f18-a5e6-778e5f7b8804,T10000580,Not Stated,Not Stated,Cotton Lycra Blend,Solid,Not Stated,Round Neck,1 T-Shirt,Not Stated,XL,Half Sleeve,Aqua,1
000636e4-07c0-5d26-adfd-f6114389edb7,EKSH001982,Not Stated,Not Stated,Cotton Blend,Checkered,Khaki,Not Stated,Not Stated,Gentle Machine Wash,Not Stated,Full Sleeve,Not Stated,1
000cb2a8-7ae7-5905-a828-9a9ebc9947b8,(GRN-KWI-ORG)-3P,Not Stated,Not Stated,Polycotton,Solid,Not Stated,Polo Neck,3 T-Shirts,Regular Machine Wash,XXL,Short Sleeve,Green;Light Green;Orange,3
000cb961-8d64-513b-896a-ab29d85fb97d,META_TSHIRT_BL-6,Not Stated,Not Stated,Pure Cotton,Graphic Print,Not Stated,Round Neck,1,Regular Machine Wash,S,Half Sleeve,Black,1
000d890b-13a7-5fdd-b740-e7136806575c,AX-531-Navy,Not Stated,Not Stated,Cotton Blend,"Striped, Checkered",Blue,Not Stated,1 Sweatshirt,Do Not Iron on Print/Embroidery/Embellishment,Not Stated,Full Sleeve,Not Stated,1
000f6224-fa1b-5771-b73d-a1f21773101c,Not Stated,Not Stated,Not Stated,Cotton,Not Stated,Multicolor,Not Stated,Not Stated,Cotton,Not Stated,Not Stated,Not Stated,Not Stated
000fe678-23a2-595b-86bf-5c6ece756471,S100-C2-Rd_Gry-,Not Stated,Not Stated,Pure Cotton,"Self Design, Solid",Not Stated,Round Neck,Not Stated,Gentle Machine Wash,L,Short Sleeve,"Red, Grey",2
001153d6-f9aa-5759-b61f-1b4fb288c686,Guruji-Blue,Not Stated,Not Stated,Pure Cotton,Printed,Not Stated,Round Neck,1 t shirt,Gentle Machine Wash,L,Short Sleeve,Blue,1
0013bc06-90e6-5f5a-872a-2f4daa082ab1,GYM WORKOUT CUT SLEEVES HOODIE,Not Stated,Not Stated,Pure Cotton,Printed,Not Stated,Hooded Neck,1,Gentle Machine Wash,XL,Sleeveless,Black,1


### NEXT UP ANALYZE PHASE